In [1]:
# %% [markdown]
# # Coletor de Odds - Versão Notebook
# 
# **Funcionalidades:**
# 1. Processa apenas eventos não coletados
# 2. Para após 3 erros 429 consecutivos
# 3. Salva progresso automaticamente
# 4. Exibe progresso em tempo real

# %%
import pandas as pd
import requests
import time
import logging
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timezone

# %% [markdown]
# ## Configurações Iniciais

# %%
# Configurações da API
TOKEN = "183604-pWN7flhoAsWGu8"
URL_ODDS = "https://api.b365api.com/v2/event/odds"

# Controle de erros 429
MAX_429_ERRORS = 3
current_429_errors = 0
stop_execution = False

# Configurar logging
logging.basicConfig(level=logger.info, format='%(asctime)s - %(levelname)s - %(message)s')

# %% [markdown]
# ## Funções Principais

# %%
def make_request(event_id):
    global current_429_errors, stop_execution
    
    params = {
        'token': TOKEN,
        'event_id': event_id
    }
    
    for attempt in range(5):
        if stop_execution:
            return None
        
        try:
            response = requests.get(URL_ODDS, params=params, timeout=15)
            
            if response.status_code == 429:
                current_429_errors += 1
                logger.warning(f"Erro 429 detectado ({current_429_errors}/{MAX_429_ERRORS})")
                
                if current_429_errors >= MAX_429_ERRORS:
                    stop_execution = True
                    logger.error("Limite de erros 429 atingido! Interrompendo...")
                    return None
                
                wait_time = (2 ** attempt) + 2
                time.sleep(wait_time)
                continue
                
            response.raise_for_status()
            return response.json()
            
        except Exception as e:
            logger.warning(f"Erro na tentativa {attempt+1}: {str(e)}")
            time.sleep(2 ** attempt)
    
    return None

# %%
def process_odds(event_id):
    global stop_execution
    
    if stop_execution:
        return None
    
    try:
        data = make_request(event_id)
        
        if not data or data.get('success') != 1:
            return None
        
        odds = data.get('results', {}).get('odds', {})
        processed = {'event_id': event_id}
        
        # Processar mercados
        for market in ['1_1', '1_2', '1_3']:
            market_data = odds.get(market, [])
            
            # Selecionar odd mais relevante
            valid_odds = [o for o in market_data if o.get('ss') == '0-0']
            if not valid_odds:
                valid_odds = [o for o in market_data if not o.get('ss')]
            
            if valid_odds:
                best_odd = max(valid_odds, key=lambda x: int(x.get('add_time', 0)))
                prefix = f"{market}_"
                
                if market == '1_3':
                    processed.update({
                        f"{prefix}over": best_odd.get('over_od'),
                        f"{prefix}under": best_odd.get('under_od'),
                        f"{prefix}handicap": best_odd.get('handicap')
                    })
                else:
                    processed.update({
                        f"{prefix}home": best_odd.get('home_od'),
                        f"{prefix}draw": best_odd.get('draw_od'),
                        f"{prefix}away": best_odd.get('away_od')
                    })
                
                # Converter timestamp
                ts = int(best_odd.get('add_time', 0))
                processed[f"{prefix}time"] = datetime.fromtimestamp(ts, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        
        return processed
    
    except Exception as e:
        logger.error(f"Erro no evento {event_id}: {str(e)}")
        return None

# %% [markdown]
# ## Carregar Dados e Executar

# %%
# Carregar datasets
try:
    df_existing = pd.read_excel('dados_com_odds_train_test.xlsx')
    existing_ids = set(df_existing['event_id'])
    logger.info(f"Carregados {len(df_existing)} eventos existentes")
except FileNotFoundError:
    df_existing = pd.DataFrame()
    existing_ids = set()
    logger.info("Arquivo não encontrado - iniciando do zero")

df_events = pd.read_excel('test_dataset.xlsx')
new_events = df_events[~df_events['event_id'].isin(existing_ids)]

if new_events.empty:
    logger.info("Nenhum novo evento para processar")
else:
    logger.info(f"Encontrados {len(new_events)} novos eventos")

# %%
# Executar coleta com progresso
results = []
batch_size = 10  # Reduza se encontrar erros 429

with tqdm(total=len(new_events), desc="Processando Odds") as pbar:
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(process_odds, row.event_id): row for _, row in new_events.iterrows()}
        
        for future in as_completed(futures):
            if stop_execution:
                executor.shutdown(wait=False, cancel_futures=True)
                break
                
            result = future.result()
            if result:
                results.append(result)
                pbar.update(1)
                
            # Salvar progresso a cada batch
            if len(results) % batch_size == 0:
                df_temp = pd.DataFrame(results)
                if not df_temp.empty:
                    df_final = pd.concat([df_existing, df_temp], ignore_index=True)
                    df_final.to_csv('dados_com_odds.csv', index=False)
                    logger.info(f"Progresso salvo - {len(df_final)} eventos")

# %% [markdown]
# ## Salvamento Final e Resultados

# %%
# Consolidar dados finais
if results:
    df_final = pd.concat([df_existing, pd.DataFrame(results)], ignore_index=True)
    df_final.to_csv('dados_com_odds.csv', index=False)
    df_final.to_excel('dados_com_odds.xlsx', index=False)
    logger.info(f"Processo completo! Total de eventos: {len(df_final)}")
    
    # Exibir amostra
    display(df_final.tail(3))
else:
    logger.info("Nenhum novo dado foi processado")

2025-04-11 23:35:23,480 - INFO - Arquivo não encontrado - iniciando do zero


KeyError: 'event_id'

In [1]:
# %% [markdown]
# # Coletor de Odds - Versão Notebook (Excel)
# 
# **Modificações principais:**
# 1. Leitura de arquivo Excel como entrada
# 2. Adição automática de novas colunas
# 3. Salvamento em Excel mantendo estrutura original
# 4. Processamento apenas de registros incompletos

# %%
import pandas as pd
import requests
import time
import logging
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timezone

# %% [markdown]
# ## Configurações Iniciais

# %%
# Configurações da API
TOKEN = "183604-pWN7flhoAsWGu8"
URL_ODDS = "https://api.b365api.com/v2/event/odds"

# Controle de erros 429
MAX_429_ERRORS = 3
current_429_errors = 0
stop_execution = False

# Lista de novas colunas
NEW_COLUMNS = [
    '1_1_home', '1_1_draw', '1_1_away', '1_1_time',
    '1_2_home', '1_2_draw', '1_2_away', '1_2_time',
    '1_3_over', '1_3_under', '1_3_handicap', '1_3_time'
]

# Configurar logging
logging.basicConfig(level=logger.info, format='%(asctime)s - %(levelname)s - %(message)s')

# %% [markdown]
# ## Funções Principais (Mesmas do Original)

# %%
def make_request(event_id):
    global current_429_errors, stop_execution
    
    params = {
        'token': TOKEN,
        'event_id': event_id
    }
    
    for attempt in range(5):
        if stop_execution:
            return None
        
        try:
            response = requests.get(URL_ODDS, params=params, timeout=15)
            
            if response.status_code == 429:
                current_429_errors += 1
                logger.warning(f"Erro 429 detectado ({current_429_errors}/{MAX_429_ERRORS})")
                
                if current_429_errors >= MAX_429_ERRORS:
                    stop_execution = True
                    logger.error("Limite de erros 429 atingido! Interrompendo...")
                    return None
                
                wait_time = (2 ** attempt) + 2
                time.sleep(wait_time)
                continue
                
            response.raise_for_status()
            return response.json()
            
        except Exception as e:
            logger.warning(f"Erro na tentativa {attempt+1}: {str(e)}")
            time.sleep(2 ** attempt)
    
    return None

# %%
def process_odds(event_id):
    global stop_execution
    
    if stop_execution:
        return None
    
    try:
        data = make_request(event_id)
        
        if not data or data.get('success') != 1:
            return None
        
        odds = data.get('results', {}).get('odds', {})
        processed = {'event_id': event_id}
        
        # Processar mercados
        for market in ['1_1', '1_2', '1_3']:
            market_data = odds.get(market, [])
            
            # Selecionar odd mais relevante
            valid_odds = [o for o in market_data if o.get('ss') == '0-0']
            if not valid_odds:
                valid_odds = [o for o in market_data if not o.get('ss')]
            
            if valid_odds:
                best_odd = max(valid_odds, key=lambda x: int(x.get('add_time', 0)))
                prefix = f"{market}_"
                
                if market == '1_3':
                    processed.update({
                        f"{prefix}over": best_odd.get('over_od'),
                        f"{prefix}under": best_odd.get('under_od'),
                        f"{prefix}handicap": best_odd.get('handicap')
                    })
                else:
                    processed.update({
                        f"{prefix}home": best_odd.get('home_od'),
                        f"{prefix}draw": best_odd.get('draw_od'),
                        f"{prefix}away": best_odd.get('away_od')
                    })
                
                # Converter timestamp
                ts = int(best_odd.get('add_time', 0))
                processed[f"{prefix}time"] = datetime.fromtimestamp(ts, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        
        return processed
    
    except Exception as e:
        logger.error(f"Erro no evento {event_id}: {str(e)}")
        return None

# %% [markdown]
# ## Carregar Dados e Executar (Ajustado para Excel)

# %%
# Carregar dataset
try:
    df_events = pd.read_excel('test_dataset.xlsx')
    
    # Adicionar colunas faltantes
    for col in NEW_COLUMNS:
        if col not in df_events.columns:
            df_events[col] = None
            
    # Identificar eventos não processados
    mask_na = df_events[NEW_COLUMNS].isna().all(axis=1)
    event_ids_to_process = df_events[mask_na]['event_id'].tolist()
    
    logger.info(f"Total de eventos: {len(df_events)}")
    logger.info(f"Eventos a processar: {len(event_ids_to_process)}")

except FileNotFoundError:
    logger.error("Arquivo 'test_dataset.xlsx' não encontrado!")
    exit()

if not event_ids_to_process:
    logger.info("Nenhum novo evento para processar")
    exit()

# %% [markdown]
# ## Execução do Processamento

# %%
results = []
batch_size = 5  # Reduzido para arquivos Excel

with tqdm(total=len(event_ids_to_process), desc="Processando Odds") as pbar:
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(process_odds, eid): eid for eid in event_ids_to_process}
        
        for future in as_completed(futures):
            if stop_execution:
                executor.shutdown(wait=False, cancel_futures=True)
                break
                
            result = future.result()
            if result:
                results.append(result)
                pbar.update(1)
                
            # Salvamento parcial
            if len(results) % batch_size == 0 and results:
                partial_df = pd.DataFrame(results)
                df_events.set_index('event_id', inplace=True)
                df_events.update(partial_df.set_index('event_id'))
                df_events.reset_index(inplace=True)
                
                try:
                    df_events.to_csv('temp_progress.csv', index=False)
                    logger.info("Progresso salvo temporariamente")
                except Exception as e:
                    logger.warning(f"Erro ao salvar progresso: {str(e)}")

# %% [markdown]
# ## Salvamento Final

# %%
if results:
    # Atualizar dataframe principal
    final_df = pd.DataFrame(results)
    df_events.set_index('event_id', inplace=True)
    df_events.update(final_df.set_index('event_id'))
    df_events.reset_index(inplace=True)
    
    # Salvar arquivo final
    output_file = 'df_merged.csv'
    df_events.to_csv(output_file, index=False)
    
    # Resultados
    logger.info(f"Processo completo! Arquivo salvo como: {output_file}")
    print("\nAmostra dos dados processados:")
    display(df_events[['event_id'] + NEW_COLUMNS].tail(3))
    
    # Limpar arquivo temporário
    try:
        os.remove('temp_progress.xlsx')
    except:
        pass
else:
    logger.info("Nenhum dado novo foi processado")

2025-04-13 10:56:06,074 - INFO - Total de eventos: 5355
2025-04-13 10:56:06,075 - INFO - Eventos a processar: 5355


Processando Odds:   0%|          | 0/5355 [00:00<?, ?it/s]

2025-04-13 10:56:07,087 - WARNING - Erro 429 detectado (1/3)
2025-04-13 10:56:07,093 - WARNING - Erro 429 detectado (2/3)
2025-04-13 10:56:07,137 - WARNING - Erro 429 detectado (3/3)
2025-04-13 10:56:07,139 - ERROR - Limite de erros 429 atingido! Interrompendo...
2025-04-13 10:56:10,096 - INFO - Nenhum dado novo foi processado


In [ ]:
# %% [markdown]
# # Coletor de Odds - Versão Final (Excel)
# 
# **Funcionalidades:**
# 1. Processamento exclusivo de arquivos Excel (.xlsx)
# 2. Adição automática de colunas necessárias
# 3. Salvamento incremental seguro
# 4. Detecção inteligente de registros incompletos

# %%
import pandas as pd
import requests
import time
import logging
import os
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timezone

# %% [markdown]
# ## Configurações Iniciais

# %%
# Configurações da API
TOKEN = "183604-pWN7flhoAsWGu8"
URL_ODDS = "https://api.b365api.com/v2/event/odds"

# Controles de execução
MAX_429_ERRORS = 3
current_429_errors = 0
stop_execution = False

# Lista de colunas de odds
COLUNAS_ODDS = [
    '1_1_home', '1_1_draw', '1_1_away', '1_1_time',
    '1_2_home', '1_2_draw', '1_2_away', '1_2_time',
    '1_3_over', '1_3_under', '1_3_handicap', '1_3_time'
]

# Configuração do logging
logging.basicConfig(
    level=logger.info,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('coletor_odds.log'),
        logging.StreamHandler()
    ]
)

# %% [markdown]
# ## Funções de Processamento

# %%
def fazer_requisicao(event_id):
    global current_429_errors, stop_execution
    
    params = {
        'token': TOKEN,
        'event_id': event_id
    }
    
    for tentativa in range(5):
        if stop_execution:
            return None
        
        try:
            resposta = requests.get(URL_ODDS, params=params, timeout=15)
            
            if resposta.status_code == 429:
                current_429_errors += 1
                logger.warning(f"Erro 429 (#{current_429_errors}/{MAX_429_ERRORS})")
                
                if current_429_errors >= MAX_429_ERRORS:
                    stop_execution = True
                    logger.error("Limite de erros 429 atingido! Abortando...")
                    return None
                
                espera = (2 ** tentativa) + 2
                time.sleep(espera)
                continue
                
            resposta.raise_for_status()
            return resposta.json()
            
        except Exception as erro:
            logger.warning(f"Tentativa {tentativa+1}/5 falhou: {str(erro)}")
            time.sleep(2 ** tentativa)
    
    return None

# %%
def processar_odds(event_id):
    global stop_execution
    
    if stop_execution:
        return None
    
def processar_odds(event_id):
    global stop_execution
    
    if stop_execution:
        return None

    # Inicializar o dicionário com valores padrão '-'
    processado = {'event_id': event_id}
    for coluna in COLUNAS_ODDS:
        processado[coluna] = '-'

    try:
        dados = fazer_requisicao(event_id)
        
        if not dados or dados.get('success') != 1:
            return processado  # Retorna com '-' preenchido

        odds = dados.get('results', {}).get('odds', {})

        for mercado in ['1_1', '1_2', '1_3']:
            dados_mercado = odds.get(mercado, [])

            odds_validas = [o for o in dados_mercado if o.get('ss') == '0-0'] or \
                           [o for o in dados_mercado if not o.get('ss')]

            if odds_validas:
                melhor_odd = max(odds_validas, key=lambda x: int(x.get('add_time', 0)))
                prefixo = f"{mercado}_"

                if mercado == '1_3':
                    processado[f"{prefixo}over"] = melhor_odd.get('over_od', '-')
                    processado[f"{prefixo}under"] = melhor_odd.get('under_od', '-')
                    processado[f"{prefixo}handicap"] = melhor_odd.get('handicap', '-')
                else:
                    processado[f"{prefixo}home"] = melhor_odd.get('home_od', '-')
                    processado[f"{prefixo}draw"] = melhor_odd.get('draw_od', '-')
                    processado[f"{prefixo}away"] = melhor_odd.get('away_od', '-')

                ts = int(melhor_odd.get('add_time', 0))
                processado[f"{prefixo}time"] = datetime.fromtimestamp(ts, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')

        return processado

    except Exception as erro:
        logger.error(f"Falha crítica no evento {event_id}: {str(erro)}")
        return processado  # Retorna com '-'


# %% [markdown]
# ## Gerenciamento de Arquivos Excel

# %%
def carregar_dataset():
    try:
        df = pd.read_csv('df_merged.csv')
        
        # Adicionar colunas faltantes
        for coluna in COLUNAS_ODDS:
            if coluna not in df.columns:
                df[coluna] = pd.NA
                
        # Identificar registros incompletos
        registros_incompletos = df[df[COLUNAS_ODDS].isna().all(axis=1)]
        ids_para_processar = registros_incompletos['event_id'].tolist()
        
        logger.info(f"Dataset carregado: {len(df)} registros")
        logger.info(f"Registros incompletos detectados: {len(ids_para_processar)}")
        
        return df, ids_para_processar
    
    except FileNotFoundError:
        logger.error("Erro: Arquivo 'test_dataset.xlsx' não encontrado!")
        exit()

# %%
def salvar_progresso(df, temp=False):
    try:
        arquivo = 'temp_progress.csv' if temp else 'df_merged.csv'
        df.to_csv(arquivo, index=False)
        logger.info(f"Progresso {'temporário ' if temp else ''}salvo em {arquivo}")
    except Exception as erro:
        logger.error(f"Erro ao salvar: {str(erro)}")

# %% [markdown]
# ## Execução Principal

# %%
# Carregar dados iniciais
df_principal, eventos = carregar_dataset()

if not eventos:
    logger.info("Nenhum registro pendente para processamento")
    exit()

# %%
# Processamento paralelo
resultados = []
lote_salvamento = 5

with tqdm(total=len(eventos), desc="Coletando Odds") as barra_progresso:
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(processar_odds, eid): eid for eid in eventos}
        
        for future in as_completed(futures):
            if stop_execution:
                executor.shutdown(wait=False, cancel_futures=True)
                break
            
            resultado = future.result()
            if resultado:
                resultados.append(resultado)
                barra_progresso.update(1)
                
                # Salvamento periódico
                if len(resultados) % lote_salvamento == 0:
                    df_atualizado = df_principal.set_index('event_id')
                    df_atualizado.update(pd.DataFrame(resultados).set_index('event_id'))
                    df_principal = df_atualizado.reset_index()
                    salvar_progresso(df_principal, temp=True)

# %% [markdown]
# ## Finalização

# %%
if resultados:
    # Atualizar dataset principal
    df_final = pd.DataFrame(resultados)
    df_principal = df_principal.set_index('event_id')
    df_principal.update(df_final.set_index('event_id'))
    df_principal = df_principal.reset_index()
    
    # Salvar resultado final
    salvar_progresso(df_principal)
    
    # Limpar temporário
    try:
        os.remove('temp_progress.xlsx')
    except:
        pass
    
    # Relatório final
    logger.info(f"Processo concluído! {len(resultados)} novos registros processados")
    print("\nVisualização dos dados:")
    display(df_principal[['event_id'] + COLUNAS_ODDS].tail(3))
else:
    logger.warning("Nenhum dado novo foi obtido")

2025-04-13 09:50:13,184 - INFO - Dataset carregado: 26884 registros
2025-04-13 09:50:13,185 - INFO - Registros incompletos detectados: 13129


Coletando Odds:   0%|          | 0/13129 [00:00<?, ?it/s]

C:\Users\Enzo\AppData\Local\Temp\ipykernel_12700\1938596793.py:214: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_atualizado.update(pd.DataFrame(resultados).set_index('event_id'))
2025-04-13 09:50:23,566 - INFO - Progresso temporário salvo em temp_progress.xlsx
2025-04-13 09:50:33,140 - INFO - Progresso temporário salvo em temp_progress.xlsx
2025-04-13 09:50:44,365 - INFO - Progresso temporário salvo em temp_progress.xlsx
2025-04-13 09:50:54,502 - INFO - Progresso temporário salvo em temp_progress.xlsx
2025-04-13 09:51:05,405 - INFO - Progresso temporário salvo em temp_progress.xlsx
2025-04-13 09:51:18,540 - INFO - Progresso temporário salvo em temp_progress.xlsx
2025-04-13 09:51:37,347 - INFO - Progresso temporário salvo em temp_progress.xlsx
2025-04-13 09:51:51,015 - INFO - Progresso tempor


Visualização dos dados:


,event_id,1_1_home,1_1_draw,1_1_away,1_1_time,1_2_home,1_2_draw,1_2_away,1_2_time,1_3_over,1_3_under,1_3_handicap,1_3_time
26881,9790071,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26882,9790070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26883,9789627,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
